In [132]:
import pprint

def pretty_print(data):
    
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(data)


In [143]:
import os

def load_zshrc_env(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('export'):
                key_value = line.strip().split(' ')[1].split('=')
                key = key_value[0]
                value = key_value[1].strip('"').strip("'")
                os.environ[key] = value

# zshrc_path = os.path.expanduser('~/.zshrc')
# load_zshrc_env(zshrc_path)

In [134]:
from langchain.document_loaders import YoutubeLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.schema import Document
from langchain.embeddings import CohereEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import gradio as gr
import json
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [135]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

directories = [
    '/Users/qinliao/learn/knausj_talon',
]

docs = []

for directory in directories:
    for dirpath, dirnames, filenames in os.walk(directory):
        for file in filenames:
            try:
                loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
                docs.extend(loader.load_and_split())
            except Exception as e:
                pass

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs)

llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo')



/Users/qinliao/.pyenv/versions/3.10.11/envs/myenv/lib/python3.10/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/qinliao/.pyenv/versions/3.10.11/envs/myenv/lib/python3.10/site-packages/langchain/llms/openai.py:687: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [136]:
# pretty_print([doc.metadata for doc in docs])
pretty_print(docs)

[   Document(page_content='2877a6849d75e5fa78c9453991a9235b4f6d9dcf\n3bf4882fa0a05b22171e59118bd7c9640aae753a\n446ec764c9caa98973eacd7f792b6a087a1b635f', metadata={'source': '/Users/qinliao/learn/knausj_talon/.git-blame-ignore-revs'}),
    Document(page_content="-\nsettings():\n    # Adjust the scale of the imgui to my liking\n    imgui.scale = 1.3\n\n    # Enable if you'd like the picker gui to automatically appear when explorer has focus\n    user.file_manager_auto_show_pickers = 0\n\n    # Set the max number of command lines per page in help\n    user.help_max_command_lines_per_page = 50\n\n    # Set the max number of contexts display per page in help\n    user.help_max_contexts_per_page = 20\n\n    # The default amount used when scrolling continuously\n    user.mouse_continuous_scroll_amount = 80\n\n    # Stop continuous scroll/gaze scroll with a pop\n    user.mouse_enable_pop_stops_scroll = 1\n\n    # Enable pop click with 'control mouse' mode.\n    # 0 = off\n    # 1 = on with ey

In [99]:

COHERE_EMBEDDINGS_DIR  = 'cohere_embeddings'

embeddings = CohereEmbeddings()
if os.path.isdir(COHERE_EMBEDDINGS_DIR):
    print("Loading embeddings from disk...")
    coherestore = Chroma(persist_directory=COHERE_EMBEDDINGS_DIR, embedding_function=embeddings)
else:
    print("Creating embeddings from documents...")
    coherestore = Chroma.from_documents(docs, embeddings, persist_directory=COHERE_EMBEDDINGS_DIR, verbose=True)
    coherestore.persist()


Using embedded DuckDB with persistence: data will be stored in: cohere_embeddings


Loading embeddings from disk...


In [100]:
query = "uppercase in talon?"
docs = coherestore.similarity_search_with_score(query)
pretty_print(docs)

[   (   Document(page_content="# knausj_talon\n\nCommunity-maintained [Talon](https://talonvoice.com/) command grammar.\n\nCan be used on its own, but shines when combined with:\n\n- [Cursorless](https://www.cursorless.org/) for programming and text editing\n- [Rango](https://github.com/david-tejada/rango) for browser navigation\n- [gaze-ocr](https://github.com/wolfmanstout/talon-gaze-ocr) for advanced cursor control using eye tracking and text recognition (OCR)\n- [AXKit](https://github.com/phillco/talon-axkit) (macOS only) to enhance Talon with native OS accessibility integrations\n- [Other user file sets](https://talon.wiki/talon_user_file_sets/)\n\n## Installation\n\n### Prerequisites\n\n- [Talon](https://talonvoice.com/)\n- Mac, Windows, or Linux\n- Can work with both Talon's built-in Conformer (wav2letter) speech recognition engine (recommended), or Dragon Naturally Speaking (Windows) / Dragon for Mac (although beware that Dragon for Mac is deprecated).\n- Includes commands for w

In [139]:
OPEN_AI_EMBEDDINGS_DIR  = 'openai_embeddings'
embeddings = OpenAIEmbeddings()
if os.path.isdir(OPEN_AI_EMBEDDINGS_DIR):
    print("Loading embeddings from disk...")
    openaistore = Chroma(persist_directory=OPEN_AI_EMBEDDINGS_DIR, embedding_function=embeddings)
else:
    print("Creating embeddings from documents...")
    openaistore = Chroma.from_documents(docs, embeddings, persist_directory=OPEN_AI_EMBEDDINGS_DIR, verbose=True)
    openaistore.persist()
    print("Done creating embeddings from documents...")

Using embedded DuckDB with persistence: data will be stored in: openai_embeddings


Creating embeddings from documents...
Done creating embeddings from documents...


In [138]:
query = "chrome focus"
docs = openaistore.similarity_search_with_score(query)
pretty_print(docs)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


NoIndexException: Index not found, please create an instance before querying

In [126]:
open_ai_chain = RetrievalQA.from_chain_type(llm, retriever=openaistore.as_retriever(), return_source_documents=True,  )
open_ai_chain({'query': 'what are the spoken commands related to the mouse?'})

{'query': 'what are the spoken commands related to the mouse?',
 'result': 'The spoken commands related to the mouse are "curse yes" and "curse no" to show or hide the mouse cursor, "wheel left", "wheel tiny left", "wheel right", "wheel tiny right" for scrolling, and "copy mouse position" to copy the current mouse position.',
 'source_documents': [Document(page_content='All formatter-related commands are defined here\nhttps://github.com/knausj85/knausj_talon/blob/main/core/text/text.talon#L8\n\n### Mouse commands\n\nSee https://github.com/knausj85/knausj_talon/blob/main/plugin/mouse/mouse.talon\n\n### Generic editing commands\n\nhttps://github.com/knausj85/knausj_talon/blob/main/core/edit/edit.talon\n\nThese generic commands are global. Commands such as `go word left` will work in any text box.\n\n### Repeating commands\n\nFor repeating commands, useful voice commands are defined here: https://github.com/knausj85/knausj_talon/blob/main/plugin/repeater/repeater.talon\n\nTry saying e.g. 

In [140]:
import json
from json import JSONEncoder
from langchain.schema import Document

class DocumentEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Document):
            return obj.__dict__
        return super(DocumentEncoder, self).default(obj)

In [129]:
openai_retriever = openaistore.as_retriever(search_kwargs={"k": 10})

def search_similarity(query):
    result = openai_retriever.get_relevant_documents(query)
    return json.dumps(result, cls=DocumentEncoder)

   

iface = gr.Interface(
    fn=search_similarity,
    inputs=gr.inputs.Textbox(lines=2, placeholder="similarity search..."),
    outputs=[
        gr.outputs.Textbox(label="Result"),
    ],
)

iface.launch()

/Users/qinliao/.pyenv/versions/3.10.11/envs/myenv/lib/python3.10/site-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/Users/qinliao/.pyenv/versions/3.10.11/envs/myenv/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Users/qinliao/.pyenv/versions/3.10.11/envs/myenv/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Users/qinliao/.pyenv/versions/3.10.11/envs/myenv/lib/python3.10/site-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


In [141]:
def search(query):
    result  =  open_ai_chain({'query': query})
    return json.dumps(result, cls=DocumentEncoder)

   

iface = gr.Interface(
    fn=search,
    inputs=gr.inputs.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=[
        gr.outputs.Textbox(label="Result"),
    ],
)

iface.launch()

/Users/qinliao/.pyenv/versions/3.10.11/envs/myenv/lib/python3.10/site-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/Users/qinliao/.pyenv/versions/3.10.11/envs/myenv/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Users/qinliao/.pyenv/versions/3.10.11/envs/myenv/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Users/qinliao/.pyenv/versions/3.10.11/envs/myenv/lib/python3.10/site-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Generally, answer with how the user would do in in talon -- not how to implement it. 
.talon files: spoken work commands that users can use to activate certain commands.
.python files: implementations of the commands that are called by the .talon files.\
.csv files: configurations and settings that users can change


{context}

Question: {question}
Answer: """
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
prompt_template.format(question=query, context=docs)

'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\nGenerally, answer with how the user would do in in talon -- not how to implement it. \n.talon files: spoken work commands that users can use to activate certain commands.\n.python files: implementations of the commands that are called by the .talon files..csv files: configurations and settings that users can change\n\n\n[(Document(page_content="# knausj_talon\\n\\nCommunity-maintained [Talon](https://talonvoice.com/) command grammar.\\n\\nCan be used on its own, but shines when combined with:\\n\\n- [Cursorless](https://www.cursorless.org/) for programming and text editing\\n- [Rango](https://github.com/david-tejada/rango) for browser navigation\\n- [gaze-ocr](https://github.com/wolfmanstout/talon-gaze-ocr) for advanced cursor control using eye tracking and text recognition (OCR)\\n- [AXKit](https://github.com/phillco/ta

In [ ]:
raw_questions = [
    "how do you switch to using chrome"
    # "how do i tell Talon to use ruby language when editing .rake files",
    # "Is there a way  to get  letters in dictation mode ? For ex for numbers, I can say 'numeral 10' and get 10 vs ten, but  I can't find a way to have it type specific letters in dictation mode. I know how to do so in command mode - thx",
    # "Is there a way to have conformer auto-capitalize names in dictation mode?",
    # "is there were way to control the level of debug information on launch?",
    # "Is there a spoken word command that allows me to hold a key? Example: hold shift while pressing right three times."
]

In [ ]:
prompt_template.format({"question": raw_questions[0]})

KeyError: 'context'

In [ ]:
for question in raw_questions:
    print('#'*20)
    print("question:", question)
    result  = vectorstore_chain({'query': question})
    print(f'answer: {result["result"]}')
    print(f'source: {result["source_documents"]}')
    print('#'*20)


####################
question: how do you switch to using chrome
answer: I'm sorry, I cannot determine the specific context in which you are asking this question. Can you please provide more information or context?
source: [Document(page_content="## Basic Usage\n\nThe examples below are just a very small selection of common commands for working with apps, tabs, media, mouse, etc that should help you be productive with Talon right away.  These are based on the [knaus_talon](https://github.com/knausj85/knausj_talon) Talon user file set (see [the earlier section](#install-a-talon-user-file-set)). These commands may vary depending on your individual setup.\n\n\n### Command history\n\n`command history`\n: Toggles the command history display, which shows the last few commands as Talon understood them; if you're not sure what Talon is doing, or if it understood you correctly, this is how to check it.\n\n\n### Show available comands\n\n`knausj_talon` has an integrated help.  It can show you a 

In [ ]:
vectorstore_chain({'query': "what are all the commands I have to do with mouse? and how can you use them?"})

{'query': 'what are all the commands I have to do with mouse? and how can you use them?',
 'result': ' You can use the `mouse_move()` command to move the mouse to a specific screen coordinate, `mouse_scroll()` to scroll the mouse, `mouse_click()` to click the left or right mouse button, and `mouse_button()` to list the mouse button words to use with the `mouse_click()` index parameter. You can also use the `mouse_wake()` command to enable zoom mouse mode, and the `mouse_hide()` command to hide the mouse cursor.',
 'source_documents': [Document(page_content='```\nimport io;old=sys.stdout;sys.stdout = io.StringIO();actions.list();clip.set_text(sys.stdout.getvalue());sys.stdout = old\n```\n\nSome of the more useful actions are:\n\n* `key(ctrl-a)` - Presses the keys within the parentheses. See the \'Add new keyboard shortcuts\' recipe below for some more info about using this action.\n* `insert("my text")` - Types in the text "my text"\n* `sleep(100ms)` - Waits for 100 milliseconds. This c

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [ ]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = [
    Tool.from_function(
        name = "Search",
        func=vectorstore_chain.__call__, #chain doesn't work anymore because of some inspection issue and there is no call function
        coroutine=vectorstore_chain.acall, #if you want to use async
        description="useful to answer factual questions"
    ),
]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm=ChatOpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)


agent_chain.run(input=" how do i tell Talon to use ruby language when editing .rake files")
print(agent_chain.run(input="how do i tell Talon to use ruby language when editing .rake files"))



> Entering new AgentExecutor chain...
{
    "action": "Search",
    "action_input": "How to set Talon to use Ruby language for .rake files"
}
Observation: {'query': 'How to set Talon to use Ruby language for .rake files', 'result': ' Talon does not require special configuration for different programming languages, but many users have per-language customizations to improve efficiency. Checkout the section on Programming Languages in the knausj Talon user file set README for more information on how to use different language modes included in knausj.', 'source_documents': [Document(page_content="# FAQ\n\n### What can I say?  \nCheckout the [common commands](/getting_started/#basic-usage) to get started using Talon.\n\n### How can I code in different languages with Talon\nTalon does not require special configuration for different programming languages, but many users have per-language customizations to improve efficiency. Checkout the section on [Programming Languages](https://github.com

'Talon does not require special configuration for different programming languages, but many users have per-language customizations to improve efficiency. Checkout the section on Programming Languages in the knausj Talon user file set README for more information on how to use different language modes included in knausj.'